In [24]:
import numpy as np

D = 784
L = 12
M = 100
a = np.random.uniform(-1, 1, D)
b = np.random.uniform(-1, 1, L)
x = np.random.randint(0, 2, (M, D))
w = np.random.uniform(-1, 1, (D, L))

"""
import numpy as np
D = len(data[1])
L = 3
a = aE[-1]
b = bE[-1]
x = data
w = wE[-1]
"""

class LOGlike(object):
    """
    FORMAT:
    D = 784      # visible
    L = 12       # hidden
    M = len(x)   # number of data points
    a = np.random.uniform(-1, 1, D)
    b = np.random.uniform(-1, 1, L)
    x = np.random.randint(0, 2, (M, D))
    z = np.random.randint(0, 2, L)
    w = np.random.uniform(-1, 1, (D, L))
    """

    def __init__(self, a, b, x, w):
        self.a  = a   # visible
        self.b  = b   # hidden
        self.D  = a.shape[0]
        self.L  = b.shape[0]
        self.xs = x
        self.w  = w
        self.M  = len(x)
        self.q  = None
        self.esit = None
        self.zs   = None
        self.make_z_states()
        #
    #####
    #
    def make_z_states(self):
        """
        2^L config
        """
        
        def get_bin(i, L):
            b = bin(i)[2:]
            return ("0" * (L - len(b))) + b
            #
        #####
        #
        self.zs = np.array([[int(bit) for bit in get_bin(i, self.L)]
                                       for i in range(2 ** self.L)])
        #
    #####
    #
    def H(self, z):
        """
        H(z) = a + w @ z
        """
        
        return self.a + np.dot(self.w, z)
        #
    #####
    #
    def G(self, z):
        """
        G(z) = exp(b @ z).
        """
        
        return np.exp(np.dot(self.b, z))
        #
    #####
    #
    def get_numerator(self, x):
        """
        ln{sum_z[G(z) * exp[dot(H_i(z) * x_i)]]}
        """
            
        H_z_matrix = np.array([self.H(z) for z in self.zs])  # Shape (2^L, D)
        G_z_vector = np.array([self.G(z) for z in self.zs])  # Shape (2^L,)

        exp_H_x    = np.exp(H_z_matrix @ x)                  # Shape (2^L,)
        numerator  = np.sum(G_z_vector * exp_H_x)
        return np.log(numerator)
        #
    #####
    #
    def get_denominator(self):
        """
        ln(Z) = D * ln(q) + ln{sum_z[G(z) * prod_i[(1 + H_i(z)) / q]]}
        """
        
        H_z_matrix = np.array([self.H(z) for z in self.zs])  # Shape (2^L, D)
        G_z_vector = np.array([self.G(z) for z in self.zs])  # Shape (2^L,)

        # q average over 1 + exp(H_i(z))
        self.q     = 1 + np.exp(np.mean(H_z_matrix))

        # sum_Z
        prod_H_q   = np.prod((1 + np.exp(H_z_matrix)) / self.q, axis = 1)  # Shape (2^L,)
        Z_value    = np.sum(G_z_vector * prod_H_q)

        return self.D * np.log(self.q) + np.log(Z_value)
        #
    #####
    #
    def run(self):
        """
        l_θ(x) = ln{sum_z[G(z) * exp(H_i(z) * x_i)]} - ln{Z}
        """
        
        lnZ             = self.get_denominator()
        log_likelihoods = np.array([self.get_numerator(x) - lnZ for x in self.xs])
        self.esit       = np.mean(log_likelihoods)
        #
    #####
    #
    def print(self):
        return self.esit
        #
    #####
    #
#####
#
#Exe
log_likehood = LOGlike(a, b, x, w)
log_likehood.run()
print("Log-likelihood media sul dataset:", log_likehood.print())

Log-likelihood media sul dataset: -814.0683441013111
